<main style="font-family: TeX Gyre Termes; font-size: 1.2rem">

### MEX #7 - Geyzson Kristoffer
SN:2023-21036

https://uvle.upd.edu.ph/mod/assign/view.php?id=547271

<hr>

# Problem #1

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tqdm as notebook_tqdm

diabetes = pd.read_csv('diabetes_data_upload.csv')
diabetes.head()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive


In [11]:
diabetes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Age                 520 non-null    int64 
 1   Gender              520 non-null    object
 2   Polyuria            520 non-null    object
 3   Polydipsia          520 non-null    object
 4   sudden weight loss  520 non-null    object
 5   weakness            520 non-null    object
 6   Polyphagia          520 non-null    object
 7   Genital thrush      520 non-null    object
 8   visual blurring     520 non-null    object
 9   Itching             520 non-null    object
 10  Irritability        520 non-null    object
 11  delayed healing     520 non-null    object
 12  partial paresis     520 non-null    object
 13  muscle stiffness    520 non-null    object
 14  Alopecia            520 non-null    object
 15  Obesity             520 non-null    object
 16  class               520 no

In [12]:
import optuna
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score
from ast import literal_eval

df = diabetes.copy()

df['class'] = df['class'].map({'Positive': 1, 'Negative': 0})
X = df.drop('class', axis=1)
y = df['class']

# 1a.
# encoding the labels and splitting the data into train and test sets
le = LabelEncoder()
X = X.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# running optuna
def objective(trial):
    classifier_name = trial.suggest_categorical('classifier', ['MLPClassifier', 'RandomForest', 'XGBoost', 'LogisticRegression', 'NaiveBayes', 'SVM', 'kNN'])
    
    if classifier_name == 'MLPClassifier':
        hidden_layer_sizes_str = trial.suggest_categorical('hidden_layer_sizes', ['(50, 50, 50)', '(50, 100, 50)', '(100,)'])
        hidden_layer_sizes = eval(hidden_layer_sizes_str)
        params = {
            'hidden_layer_sizes': hidden_layer_sizes,
            'activation': trial.suggest_categorical('activation', ['tanh', 'relu']),
            'solver': trial.suggest_categorical('solver', ['sgd', 'adam']),
            'max_iter': trial.suggest_int('max_iter', 1000, 2000), 
            'batch_size': trial.suggest_int('batch_size', 1, 100),
            'learning_rate_init': trial.suggest_float('learning_rate_init', 1e-5, 1e-2),
            'alpha': trial.suggest_float('alpha', 1e-5, 1e-2),
            'shuffle': trial.suggest_categorical('shuffle', [True, False]),
            'tol': trial.suggest_float('tol', 1e-5, 1e-2),
            'momentum': trial.suggest_float('momentum', 1e-5, 1e-2),
            'early_stopping': trial.suggest_categorical('early_stopping', [True, False]),
        }
        model = MLPClassifier(**params)
        
    elif classifier_name == 'RandomForest':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
            'max_depth': trial.suggest_int('max_depth', 1, 50),
            'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 14),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 14),
            'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),

        }
        model = RandomForestClassifier(**params)
        
    elif classifier_name == 'XGBoost':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
            'max_depth': trial.suggest_int('max_depth', 1, 50),
            'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0),
            'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0),
            'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1e-6, 1e6),

        }
        model = xgb.XGBClassifier(**params)
        
    elif classifier_name == 'LogisticRegression':
        params = {
            'C': trial.suggest_float('C', 1e-8, 1e2),
            'max_iter': trial.suggest_int('max_iter', 1000, 2000),
        }
        model = LogisticRegression(**params)
        
    elif classifier_name == 'NaiveBayes':
        params = {
            'var_smoothing': trial.suggest_float('var_smoothing', 1e-8, 1e-2),
        }
        model = GaussianNB(**params)
        
    elif classifier_name == 'SVM':
        params = {
            'C': trial.suggest_float('C', 1e-8, 1e2),
            'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
            'degree': trial.suggest_int('degree', 1, 10),
            'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
        }
        model = SVC(**params)
        
    elif classifier_name == 'kNN':
        params = {
            'n_neighbors': trial.suggest_int('n_neighbors', 1, 50),
            'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
            'algorithm': trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute']),
            'leaf_size': trial.suggest_int('leaf_size', 1, 50),
        }
        model = KNeighborsClassifier(**params)

    f1_scorer = make_scorer(f1_score, average='weighted')
    precision_scorer = make_scorer(precision_score)
    recall_scorer = make_scorer(recall_score)

    score = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=10).mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-01-23 04:13:03,199] A new study created in memory with name: no-name-373802b0-5de5-46d8-a3ba-862fc63116dc


[I 2024-01-23 04:13:04,083] Trial 0 finished with value: 0.9179442508710803 and parameters: {'classifier': 'RandomForest', 'n_estimators': 714, 'max_depth': 13, 'criterion': 'entropy', 'min_samples_split': 10, 'min_samples_leaf': 7, 'bootstrap': True}. Best is trial 0 with value: 0.9179442508710803.
[I 2024-01-23 04:13:04,111] Trial 1 finished with value: 0.870150987224158 and parameters: {'classifier': 'kNN', 'n_neighbors': 28, 'weights': 'uniform', 'algorithm': 'kd_tree', 'leaf_size': 8}. Best is trial 0 with value: 0.9179442508710803.
[I 2024-01-23 04:13:04,124] Trial 2 finished with value: 0.92061556329849 and parameters: {'classifier': 'LogisticRegression', 'C': 27.595157339340947, 'max_iter': 1996}. Best is trial 2 with value: 0.92061556329849.
[I 2024-01-23 04:13:09,391] Trial 3 finished with value: 0.5986062717770034 and parameters: {'classifier': 'XGBoost', 'n_estimators': 334, 'max_depth': 24, 'learning_rate': 0.0012840005839864727, 'booster': 'dart', 'reg_alpha': 0.993731853

In [13]:
best_trial = study.best_trial
print(f"Best trial final score: {best_trial.value}")
for key, value in best_trial.params.items():
    print(f"{key}: {value}")

best_classifier = best_trial.params['classifier']

Best trial final score: 0.9734610917537747
classifier: SVM
C: 2.9090105588450754
kernel: poly
degree: 5
gamma: scale


In [14]:
best_params = study.best_trial.params
best_params

{'classifier': 'SVM',
 'C': 2.9090105588450754,
 'kernel': 'poly',
 'degree': 5,
 'gamma': 'scale'}

In [23]:
if best_params['classifier'] == 'MLPClassifier':
    model = MLPClassifier(**{k: v for k, v in best_params.items() if k != 'classifier'})
elif best_params['classifier'] == 'RandomForest':
    model = RandomForestClassifier(**{k: v for k, v in best_params.items() if k != 'classifier'})
elif best_params['classifier'] == 'XGBoost':
    model = xgb.XGBClassifier(**{k: v for k, v in best_params.items() if k != 'classifier'})
elif best_params['classifier'] == 'LogisticRegression':
    model = LogisticRegression(**{k: v for k, v in best_params.items() if k != 'classifier'})
elif best_params['classifier'] == 'NaiveBayes':
    model = GaussianNB()
elif best_params['classifier'] == 'SVM':
    model = SVC(**{k: v for k, v in best_params.items() if k != 'classifier'})
elif best_params['classifier'] == 'kNN':
    model = KNeighborsClassifier(**{k: v for k, v in best_params.items() if k != 'classifier'})

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print('The accuracy of the and F1-score of the best model in the test data ')
print(f"Test Set Evaluation Metrics:\nAccuracy: {accuracy}\nF1 Score: {f1}\nPrecision: {precision}\nRecall: {recall}")


The accuracy of the and F1-score of the best model in the test data 
Test Set Evaluation Metrics:
Accuracy: 0.9807692307692307
F1 Score: 0.980602297008547
Precision: 0.9812961011591148
Recall: 0.9807692307692307


In [24]:
def objective2(trial):

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 50),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 14),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 14),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),

    }
    model = RandomForestClassifier(**params)

    f1_scorer = make_scorer(f1_score, average='weighted')
    precision_scorer = make_scorer(precision_score)
    recall_scorer = make_scorer(recall_score)

    score = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=10, scoring=f1_scorer).mean()
    return score

study2 = optuna.create_study(direction='maximize')
study2.optimize(objective2, n_trials=100)

best_trial2 = study2.best_trial
print(f"Best trial final score: {best_trial2.value}")
for key, value in best_trial2.params.items():
    print(f"{key}: {value}")


[I 2024-01-23 04:17:31,795] A new study created in memory with name: no-name-041dc1fb-a486-4389-b543-b6fe4a02ac25
[I 2024-01-23 04:17:31,914] Trial 0 finished with value: 0.9468575229369947 and parameters: {'n_estimators': 69, 'max_depth': 16, 'criterion': 'entropy', 'min_samples_split': 13, 'min_samples_leaf': 5, 'bootstrap': False}. Best is trial 0 with value: 0.9468575229369947.
[I 2024-01-23 04:17:32,922] Trial 1 finished with value: 0.8937037041242073 and parameters: {'n_estimators': 969, 'max_depth': 1, 'criterion': 'gini', 'min_samples_split': 7, 'min_samples_leaf': 13, 'bootstrap': True}. Best is trial 0 with value: 0.9468575229369947.
[I 2024-01-23 04:17:33,295] Trial 2 finished with value: 0.915081379664103 and parameters: {'n_estimators': 400, 'max_depth': 21, 'criterion': 'entropy', 'min_samples_split': 2, 'min_samples_leaf': 14, 'bootstrap': False}. Best is trial 0 with value: 0.9468575229369947.
[I 2024-01-23 04:17:33,429] Trial 3 finished with value: 0.9420767735497775 a

Best trial final score: 0.9831326497607644
n_estimators: 209
max_depth: 27
criterion: entropy
min_samples_split: 6
min_samples_leaf: 1
bootstrap: False


In [25]:
best_classifier2 = best_trial2.params
best_classifier2

{'n_estimators': 209,
 'max_depth': 27,
 'criterion': 'entropy',
 'min_samples_split': 6,
 'min_samples_leaf': 1,
 'bootstrap': False}

In [26]:
model2 = RandomForestClassifier(**best_classifier2)
model2

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=27,
                       min_samples_split=6, n_estimators=209)

In [31]:
model2.fit(X_train, y_train)

y_pred2 = model2.predict(X_test)
accuracy2 = accuracy_score(y_test, y_pred2)
f1_2 = f1_score(y_test, y_pred2, average='weighted')
precision2 = precision_score(y_test, y_pred2, average='weighted')
recall2 = recall_score(y_test, y_pred2, average='weighted')

print('The F1-Score of a better RF model in the test data is \n', f1_2)
print()
print(f"Test Set Evaluation Metrics:\nAccuracy: {accuracy2}\nF1 Score: {f1_2}\nPrecision: {precision2}\nRecall: {recall2}")


The F1-Score of a better RF model in the test data is 
 0.9904222748776574

Test Set Evaluation Metrics:
Accuracy: 0.9903846153846154
F1 Score: 0.9904222748776574
Precision: 0.9906674208144797
Recall: 0.9903846153846154
